### **_Importar librerias_**

In [705]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [706]:
pd.set_option("display.max_columns",None)

In [707]:
def crear_columnas(df,nueva_col,valor_busqueda):
   # df[nueva_col]= np.nan
    for index, row in df.iterrows():
     for col in df.columns[:-1]:
        if pd.notna(row[col]):
            for valor in valor_busqueda:
                if valor in str(row[col]):
                    df.at[index, nueva_col] =valor 
                    #df.at[index, col]= np.nan
                    break





# Función para crear y rellenar la nueva columna en base a un patrón
def crear_columnas_conpatron(df, nueva_col, patron, lborrar):
    df[nueva_col] = np.nan  # Inicializar la nueva columna con valores NaN si no existe
    for index, row in df.iterrows():
        for col in df.columns[:-1]:  # Excluir la nueva columna en la iteración
            if pd.notna(row[col]):
                valor = str(row[col])
                if pd.Series(valor).str.contains(patron, regex=True).any():
                    if pd.isna(df.at[index, nueva_col]):  # Verificar si la nueva columna es NaN
                        valorasignar = pd.Series(valor).str.replace(":", "").str.replace(patron, "", regex=True).values[0]
                        if valorasignar == "" or not lborrar:
                            df.at[index, nueva_col] = valor.strip()
                        else:
                            df.at[index, nueva_col] = valorasignar.strip()
                    break  # Salir del bucle interno una vez que se ha encontrado el valor y asignado




In [708]:
import pandas as pd
import numpy as np

# Función para crear y rellenar la nueva columna en base a un patrón
def crear_columnas_conpatron(df, nueva_col, patron):
    df[nueva_col] = np.nan  # Inicializar la nueva columna con valores NaN si no existe
    for index, row in df.iterrows():
        #for col in df.columns[:-1]:  # Excluir la nueva columna en la iteración
        for col in df.columns:
            if pd.notna(row[col]):
                valor = str(row[col])
                if pd.Series(valor).str.contains(patron, regex=True).any():
                    df.at[index, nueva_col] = row[col]
                    # df.at[index, col] = np.nan  # Descomentar esta línea si quieres establecer NaN en el valor original
                    break  # Salir del bucle interno una vez que se ha encontrado el valor y asignado



In [709]:

# Función para buscar en una columna específica y crear una nueva columna
def crear_columnas_conpatron_col(df, col_buscar, nueva_col, patron):
    df[nueva_col] = 0  # Inicializar la nueva columna con valores 0
    for index, row in df.iterrows():
        if pd.notna(row[col_buscar]):
            valor = str(row[col_buscar])
            if pd.Series(valor).str.contains(patron, regex=True).any():
                df.at[index, nueva_col] = 1  # Asignar 1 si el patrón se encuentra



In [710]:
# Definir la función replaceporvacio
def replaceporvacio(df, col, palabra_a_sustituir):
    df[col] = df[col].replace(palabra_a_sustituir, "", regex=True)

### **_Lectura de datos_**

In [711]:
redpiso_normal= pd.read_excel("../01-datos-iniciales/06-RedPiso/Redpiso.xlsx")
redpiso_detalles= pd.read_excel("../01-datos-iniciales/06-RedPiso/Detalle_RedPiso.xlsx")
dbcp_ordenado= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [712]:
dbcp_ordenado= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')
#dbcp_ordenado= pd.read_excel("distr_barrios_codpostal.xlsx")
#redpiso_normal= pd.read_excel("Redpiso.xlsx")
#redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [713]:
# Bien aqui vamos a construir una funcion que recorra todo el dataframe y tome los caracteres especiales 
# las comas, los puntos, entre otros y los reemplaza con nada o sea basicamente eliminarlos. 

#def limpiar_texto(x):
 #   return re.sub(r'[^\w\s]','',unidecode(str(x)))



In [714]:
#def limpiar_valores(valor):
 #   if type(valor) == str:
  #      valor = re.sub(r'[^\w\s]','', valor)
   #     valor = valor.strip()
    #return valor 

In [715]:
#def normalize_string(s):
 #  if pd.notna(s):
  #      s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
     #   s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
      #  s = s.lower()  # Convertir a minúsculas
    #return s

----------

In [716]:
## Aqui empieza el primer tratamiento de redpiso normal

In [717]:
redpiso_normal.columns

Index(['Título', 'propertylistoptionsitememblem', 'propertylistoptionsitem',
       'dormitorios', 'banos', 'itemlink_URL', 'itemlink', 'officelink_URL',
       'officelink'],
      dtype='object')

In [718]:
redpiso_normal["itemlink_URL"].value_counts(dropna=False)

itemlink_URL
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pinzon-vista-alegre-carabanchel-madrid-madrid-RP302023121517                                                            2
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-benimamet-san-cristobal-villaverde-madrid-madrid-RP2842024125161                                                        1
https://www.redpiso.es/inmueble/piso-en-venta-en-pilar-fuencarral-el-pardo-madrid-madrid-RP462023121637                                                                        1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alcala-de-guadaira-portazgo-puente-de-vallecas-madrid-madrid-RP082023121205                                             1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-sierra-de-cuerda-larga-nueva-numancia-puente-de-vallecas-madrid-madrid-RP022022105196                                   1
                                                                                                      

In [719]:
# Extraer la última palabra después del guion 

redpiso_normal["referencia"] = redpiso_normal['itemlink_URL'].apply(lambda x: re.findall(r'[^-]+$', x)[0])

In [720]:
# Conversion de letras a minusculas en las columnas
redpiso_normal.columns= redpiso_normal.columns.str.lower()

In [721]:
# Columnas que seran eliminadas ya previamente analizadas.
#columnas_a_eliminar = ["item","item1","imagen4","propertylistoptionsitem3","número5"]
#eliminadas en fichero
#redpiso_normal= redpiso_normal.drop(columns=columnas_a_eliminar)

In [722]:
redpiso_normal.head(5)

,título,propertylistoptionsitememblem,propertylistoptionsitem,dormitorios,banos,itemlink_url,itemlink,officelink_url,officelink,referencia
0,106.500 €,RESERVADO,64 m²,2 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en CALLE BENIMAMET, San Cristóba...",https://www.redpiso.es/oficina/inmobiliaria-sa...,REDPISO SAN CRISTOBAL,RP2842024125161
1,119.000 €,NaN,30 m²,1 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Usera, Madrid, Madrid",https://www.redpiso.es/oficina/inmobiliaria-us...,REDPISO USERA-ORCASITAS,RP782024130863
2,120.000 €,RESERVADO,37 m²,1 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Buenavista, Carabanchel, Madr...",https://www.redpiso.es/oficina/inmobiliaria-pa...,REDPISO PAU CARABANCHEL,RP442024126007
3,120.000 €,RESERVADO,33 m²,1 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Buenavista, Carabanchel, Madr...",https://www.redpiso.es/oficina/inmobiliaria-pa...,REDPISO PAU CARABANCHEL,RP442024126427
4,127.500 €,RESERVADO,58 m²,3 hab.,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en CALLE BENIFERRI, San Cristóba...",https://www.redpiso.es/oficina/inmobiliaria-sa...,REDPISO SAN CRISTOBAL,RP2842024130186


In [723]:
redpiso_normal.columns

Index(['título', 'propertylistoptionsitememblem', 'propertylistoptionsitem',
       'dormitorios', 'banos', 'itemlink_url', 'itemlink', 'officelink_url',
       'officelink', 'referencia'],
      dtype='object')

In [724]:
# Renombramiento de la columna.
redpiso_normal = redpiso_normal.rename(columns={"título": "precio"})
#redpiso_normal = redpiso_normal.rename(columns={"imagen": "url_imagen"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitememblem": "reservado"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem": "m2_constr"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem2": "dormitorios"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink_url": "url"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink": "title"})
#redpiso_normal = redpiso_normal.rename(columns={"número": "contacto"})
redpiso_normal = redpiso_normal.rename(columns={"officelink": "nombre_oficina"})

In [725]:
redpiso_normal["precio"].value_counts(dropna=False)

precio
A consultar    640
280.000 €       21
185.000 €       20
230.000 €       19
180.000 €       18
              ... 
182.500 €        1
187.050 €        1
1.849.000 €      1
1.380.000 €      1
1.990.000 €      1
Name: count, Length: 482, dtype: int64

In [726]:
# Eliminar las filas donde el precio es "A consultar"
redpiso_normal = redpiso_normal[redpiso_normal['precio'] != 'A consultar']

In [727]:
#for col in redpiso_normal.columns: 
 #   if col != 'url': 
  #      redpiso_normal[col] = redpiso_normal[col].apply(limpiar_valores) 
     

In [728]:
# Rellena los valores nulos a sin_reserva
#redpiso_normal["reservado/sin_reserva"]= redpiso_normal["reservado/sin_reserva"].fillna("sin_reserva")

In [729]:
# Expresion regular que recorre la columna extrae los numeros de la cada valor y elimina lo demas.aplicada a la columna que se muestra.

redpiso_normal["dormitorios"] = redpiso_normal["dormitorios"].str.extract(r'(\d+)')

redpiso_normal["m2_constr"] = redpiso_normal["m2_constr"].str.extract(r'(\d+)')
# Asignar 1 a "Reservado", 0 a "Sin Reserva" y dejar el resto como están
redpiso_normal["reservado"] = redpiso_normal["reservado"].apply(lambda x: 1 if str(x).strip().upper() == "RESERVADO" else 0)

In [730]:
redpiso_normal.isna().sum()

precio             0
reservado          0
m2_constr          3
dormitorios        2
banos             34
url                0
title              0
officelink_url     0
nombre_oficina     0
referencia         0
dtype: int64

In [731]:
redpiso_normal ["precio"]= redpiso_normal["precio"].str.replace('€',"")

In [732]:
col_tiponuevo= ["precio","m2_constr","dormitorios","banos"]

redpiso_normal = redpiso_normal.replace("nan",0)
redpiso_normal[col_tiponuevo]=redpiso_normal[col_tiponuevo].fillna(0)

# Eliminar los separadores de miles solo para los valores mayores a 1.000.000
redpiso_normal['precio'] = redpiso_normal['precio'].apply(lambda x: x.replace('.', '') if float(x.replace('.', '')) >= 1000000 else x)
redpiso_normal[col_tiponuevo] = redpiso_normal[col_tiponuevo].astype("float")

redpiso_normal[col_tiponuevo]=redpiso_normal[col_tiponuevo].replace(0,np.nan)

In [733]:
redpiso_normal["web"]="redpiso"
redpiso_normal["fecha_descarga"]="28/10/2024"

In [734]:
redpiso_normal.shape

(1468, 12)

### **_Aqui se termina el tratamiento de redpiso_normal,ahora sigue el procesmiento de redpiso_detalles_**

In [735]:
redpiso_detalles.head(3)

,Título,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Tiempo,Tiempo1,Campo7,Campo8,Campo9,Campo10,Campo11,Campo12,Campo13,Tiempo2,Campo14,Campo15,Tiempo3,Campo16,Campo17,Campo18,Tiempo4,Campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500 €,"Piso en venta en CALLE BENIMAMET, San Cristóba...",ESTUPENDA VIVIENDA EN CALLE BENIMAMET,RP2842024125161,"Piso en venta en CALLE BENIMAMET, San Cristóba...",REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,2.0,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,"Piso en venta en Usera, Madrid, Madrid\n ...",ESTUDIO JARDINES DE YBARRA II,RP782024130863,"Piso en venta en Usera, Madrid, Madrid",REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,1.0,1.0,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,Metros: 30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,"Piso en venta en Buenavista, Carabanchel, Madr...",¡Tu nuevo hogar te espera en Buena Vista!,RP442024126007,"Piso en venta en Buenavista, Carabanchel, Madr...",REF: RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,1.0,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur


In [736]:
# eliminacion de columnas
#redpiso_detalles= redpiso_detalles.drop("descripcion", axis=1)


redpiso_detalles = redpiso_detalles.drop("Campo5", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo9",axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo10",axis=1)
#redpiso_detalles = redpiso_detalles.drop ("Campo11", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo12",axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo8", axis= 1)
redpiso_detalles = redpiso_detalles.drop("Campo3", axis= 1)
redpiso_detalles = redpiso_detalles.drop("Campo6", axis= 1)

In [737]:
# renombramiento de columnas 
redpiso_detalles = redpiso_detalles.rename( columns={"Campo1": "precio"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo2":"descripcion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo":"cantidad_visitas"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo7":"m2_constr"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo4": "referencia"})
redpiso_detalles = redpiso_detalles.rename( columns={"Título": "title"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo12":"m2_constr"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo13":"m2_utiles"})
#redpiso_detalles = redpiso_detalles.rename(columns={"tiempo2": "dormitorios"})
#redpiso_detalles = redpiso_detalles.rename(columns={"Campo9": "banos"})
#redpiso_detalles = redpiso_detalles.rename(columns={"campo15": "cant_armarios"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo14":"calidad_energetica"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo10":"titularanuncio"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo11":"descripcion_detallada"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo4":"referencia"})

In [738]:
# Eliminar las filas donde el precio es "A consultar"
redpiso_detalles = redpiso_detalles[redpiso_detalles['precio'] != 'A consultar']
redpiso_detalles ["precio"]= redpiso_detalles["precio"].str.replace('€',"")

In [739]:
# Columnas a minusculas
redpiso_detalles.columns = redpiso_detalles.columns.str.lower()

In [740]:
redpiso_detalles.isna().sum()

title                  0
precio                 0
descripcion            0
referencia             0
cantidad_visitas       0
fecha_publicacion      0
m2_constr              2
campo9                25
campo10              498
campo11               11
campo12                0
campo13                2
tiempo2                4
campo14                5
campo15                0
tiempo3                0
campo16                9
campo17               28
campo18               48
tiempo4              111
campo19               77
dtype: int64

In [741]:
redpiso_detalles

,title,precio,descripcion,referencia,cantidad_visitas,fecha_publicacion,m2_constr,campo9,campo10,campo11,campo12,campo13,tiempo2,campo14,campo15,tiempo3,campo16,campo17,campo18,tiempo4,campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500,"Piso en venta en CALLE BENIMAMET, San Cristóba...",RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000,"Piso en venta en Usera, Madrid, Madrid\n ...",RP782024130863,Nº Visitas 714,02/10/2024,30 m²,1.0,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,Metros: 30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000,"Piso en venta en Buenavista, Carabanchel, Madr...",RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur
3,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000,"Piso en venta en Buenavista, Carabanchel, Madr...",RP442024126427,Nº Visitas 2924,10/05/2024,33 m²,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 33 m²,Metros útiles: 30 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 145 KW h/m² año,Emisiones: 24 Kg CO2/m² año,Estado: Perfecto Estado,Agua caliente: Individual,Orientación: Oeste
4,"Piso en venta en CALLE BENIFERRI, San Cristóba...",127.500,"Piso en venta en CALLE BENIFERRI, San Cristóba...",RP2842024130186,Nº Visitas 485,12/09/2024,58 m²,1.0,VIVIENDA EN CALLE BENIFERRI,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 58 m²,Metros útiles: 50 m²,Dormitorios: 3,Exterior,Estado: Perfecto Estado,Baños: 1,Ascensor,Aire acondicionado,Agua caliente: Gas natural,Tipo de suelo: Terrazo,Calefacción: Gas Natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,"Piso en venta en CALLE NARVAEZ, Retiro, Madrid...",1.450.000,"Piso en venta en CALLE NARVAEZ, Retiro, Madrid...",RP962024124554,Nº Visitas 376,11/03/2024,158 m²,3.0,"Piso en venta en CALLE NARVAEZ, Retiro, Madrid...",REDPISO RETIRO pone a su disposición este piso...,Metros: 158 m²,Metros útiles: 129 m²,Dormitorios: 3,Garajes: 1,Armarios: 3,Baños: 3,Estado: Perfecto Estado,Orientación: Este,Exterior,Trastero,Ascensor
2091,"Piso en venta en CALLE FERNÁNDEZ DE LA HOZ, Al...",1.990.000,"Piso en venta en CALLE FERNÁNDEZ DE LA HOZ, Al...",RP1922024124579,Nº Visitas 443,11/03/2024,317 m²,3.0,NaN,En pleno barrio de Almagro. Vivienda para refo...,Metros: 317 m²,Metros útiles: 181 m²,Dormitorios: 4,Garajes: 2,Armarios: 7,Baños: 3,Estado: A reformar,Orientación: Noroeste,Exterior,Trastero,Ascensor
2095,"Piso en venta en Aluche, Latina, Madrid, Madrid",110.000,"Piso en venta en Aluche, Latina, Madrid, Madri...",RP272023121667,Nº Visitas 864,13/12/2023,68 m²,1.0,NaN,REDPISO ALUCHE CAMARENA vende en exclusiva una...,Metros: 68 m²,Metros útiles: 66 m²,Dormitorios: 2,Orientación: Este,Estado: A reformar,Baños: 1,Tipo de suelo: Parquet,Antigüedad: Más de 50 años,Año de construcción: 1965,Número de terrazas: 1,Plantas: 5
2096,"Piso en venta en CALLE FONDON, Apóstol Santiag...",170.000,"Piso en venta en CALLE FONDON, Apóstol Santiag...",RP2532022104355,Nº Visitas 2475,19/09/2022,107 m²,2.0,NaN,¡Excelente Oportunidad! Piso en venta que cuen...,Metros: 107 m²,Metros 

In [742]:
# Eliminacion de las primeras 3 letras en la referencia web - REF- dejando solo la referencia 
#redpiso_detalles["referencia_web"] = redpiso_detalles["referencia_web"].str[3:].str.strip()

In [743]:
# Reemplace todas las letras ñ por n.
#redpiso_detalles = redpiso_detalles.apply(lambda col : col.str.replace("ñ","n")if col.dtype == "object" else col)



#  Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
#redpiso_detalles = redpiso_detalles.apply(lambda col: col.apply(limpiar_valores))


# Cree una funcion que elimina todos los acentos  y caracteres especiales
#redpiso_detalles = redpiso_detalles.apply(lambda x: x.apply(lambda y: unidecode(y) if isinstance(y,str) else y )if x.dtype == "object" else x)



In [744]:
#valores_buscar= ["Dormitorios 1", "Dormitorios 2", "Dormitorios 3", "Dormitorios 4","Dormitorios1", "Dormitorios2","Dormitorios3", "Dormitorios4",
               #   "Dormitorios 5", "Dormitorios 6", "Dormitorios 7", "Dormitorios 8","Dormitorios  8", "Dormitorios 9", "Dormitorios 10", "Dormitorios5","Dormitorios6",
                #  "Dormitorios7", "Dormitorios8", "Dormitoros9"]

#crear_columnas(redpiso_detalles,"dormitorios",valores_buscar)

#valores_buscar= ["Banos 1", "Banos 2", "Banos 3", "Banos 4","Banos 5","Banos 6","Banos 7","Banos 8",
 #                  "Banos1", "Banos2", "Banos3", "Banos4", "Banos5", "Banos6", "Banos7","Banos8"]
#crear_columnas(redpiso_detalles,"banos",valores_buscar)



valores_buscar = [
    "primera planta", "Primera planta", "1ª planta", "1a planta",
    "segunda planta", "Segunda planta", "2ª planta", "2a planta",
    "tercera planta", "Tercera planta", "3ª planta", "3a planta",
    "cuarta planta", "Cuarta planta", "4ª planta", "4a planta",
    "quinta planta", "Quinta planta", "5ª planta", "5a planta",
    "sexta planta", "Sexta planta", "6ª planta", "6a planta",
    "séptima planta", "Séptima planta", "7ª planta", "7a planta",
    "octava planta", "Octava planta", "8ª planta", "8a planta",
    "novena planta", "Novena planta", "9ª planta", "9a planta",
    "décima planta", "Décima planta", "10ª planta", "10a planta",
    "undécima planta", "Undécima planta", "11ª planta", "11a planta",
    "duodécima planta", "Duodécima planta", "12ª planta", "12a planta",
    "decimotercera planta", "Decimotercera planta", "13ª planta", "13a planta",
    "decimocuarta planta", "Decimocuarta planta", "14ª planta", "14a planta",
    "decimoquinta planta", "Decimoquinta planta", "15ª planta", "15a planta",
    "decimosexta planta", "Decimosexta planta", "16ª planta", "16a planta",
    "decimoséptima planta", "Decimoséptima planta", "17ª planta", "17a planta",
    "decimoctava planta", "Decimoctava planta", "18ª planta", "18a planta",
    "decimonovena planta", "Decimonovena planta", "19ª planta", "19a planta",
    "vigésima planta", "Vigésima planta", "20ª planta", "20a planta"
]



crear_columnas(redpiso_detalles, "planta", valores_buscar)

In [745]:
redpiso_detalles.columns

Index(['title', 'precio', 'descripcion', 'referencia', 'cantidad_visitas',
       'fecha_publicacion', 'm2_constr', 'campo9', 'campo10', 'campo11',
       'campo12', 'campo13', 'tiempo2', 'campo14', 'campo15', 'tiempo3',
       'campo16', 'campo17', 'campo18', 'tiempo4', 'campo19', 'planta'],
      dtype='object')

In [746]:
col_buscar='campo11'

patron = r'(?i)^baños:' 
crear_columnas_conpatron(redpiso_detalles,"banos",patron)
#patron = r'(?i)^baño:' 
#crear_columnas_conpatron(redpiso_detalles,"banos",patron)
patron = r'(?i)^dormitorios' 
crear_columnas_conpatron(redpiso_detalles,"dormitorios",patron)
patron = r'(?i)^armarios' 
crear_columnas_conpatron(redpiso_detalles,"armarios",patron)
#patron = r'(?i)^armario' 
#crear_columnas_conpatron(redpiso_detalles,"armarios",patron)

patron = r'(?i)^consumo' 
crear_columnas_conpatron(redpiso_detalles,"consumo_energetico",patron)

patron = r'(?i)^emisiones' 
crear_columnas_conpatron(redpiso_detalles,"emisiones_CO2",patron)

#patron = r'(?i)^cal' 
#crear_columnas_conpatron(redpiso_detalles,"cal_energetica",patron)
patron = r'(?i)^reformado' 
crear_columnas_conpatron(redpiso_detalles,"reformado",patron)
patron = r'(?i)^obra nueva' 

crear_columnas_conpatron(redpiso_detalles,"obra_nueva",patron)
patron = r'(?i)^estado' 
crear_columnas_conpatron(redpiso_detalles,"estado",patron)

patron = r'(?i)^ascensor:' 
crear_columnas_conpatron(redpiso_detalles,"ascensor",patron)
#crear_columnas_conpatron(redpiso_detalles,"ascensor",patron)
patron = r'(?i)^sin ascensor' 
crear_columnas_conpatron(redpiso_detalles,"sin_ascensor",patron)
patron = r'(?i)^aire acondicionado' 
crear_columnas_conpatron(redpiso_detalles,"aire_acondicionado",patron)
patron = r'(?i)^orientación:'
crear_columnas_conpatron(redpiso_detalles,"orientacion",patron)
patron = r'(?i)^Año de construcción:'
crear_columnas_conpatron(redpiso_detalles,"ano_construccion",patron)
patron = r'(?i)^Antigüedad' 
crear_columnas_conpatron(redpiso_detalles,"antiguedad",patron)

patron = r'(?i)^agua caliente' 
crear_columnas_conpatron(redpiso_detalles,"agua_caliente",patron)

patron = r'(?i)^calefaccion' 
crear_columnas_conpatron(redpiso_detalles,"calefaccion",patron)

patron = r'(?i)^tipo de suelo' 
crear_columnas_conpatron(redpiso_detalles,"tipo_suelo",patron)

patron = r'(?i)^tipo de fachada' 
crear_columnas_conpatron(redpiso_detalles,"tipo_fachada",patron)



patron = r'(?i)^garaje' 
crear_columnas_conpatron(redpiso_detalles,"garaje",patron)


patron = r'(?i)^Metros útiles:' 
crear_columnas_conpatron(redpiso_detalles,"m2_utiles",patron)

patron = r'(?i)^exterior' 
crear_columnas_conpatron(redpiso_detalles,"exterior",patron)
patron = r'(?i)^interior' 
crear_columnas_conpatron(redpiso_detalles,"interior",patron)

patron = r'(?i)^trastero' 
crear_columnas_conpatron(redpiso_detalles,"trastero",patron)

patron = r'(?i)^piscina'
crear_columnas_conpatron(redpiso_detalles,"piscina",patron)

patron = r'(?i)^zonas verdes'
crear_columnas_conpatron(redpiso_detalles,"zonas",patron)

patron = r'(?i)^comunidad'
crear_columnas_conpatron(redpiso_detalles,"gastos_comunidad",patron)

patron = r'(?i)^terraza'
#crear_columnas_conpatron(redpiso_detalles,"terraza",patron)
crear_columnas_conpatron(redpiso_detalles, "terraza", patron)
patron = r'(?i)^balcon'
#crear_columnas_conpatron(redpiso_detalles,"terraza",patron)
crear_columnas_conpatron(redpiso_detalles, "balcon", patron)
patron = r'(?i)^jardin'
#crear_columnas_conpatron(redpiso_detalles,"terraza",patron)
crear_columnas_conpatron(redpiso_detalles, "jardin", patron)

patron = r'(?i)^minusvalido'
crear_columnas_conpatron(redpiso_detalles,"acceso_minusvalido",patron)
patron = r'(?i)^blindada'
crear_columnas_conpatron(redpiso_detalles,"puerta_blindada",patron)
patron = r'(?i)^alarma'
crear_columnas_conpatron(redpiso_detalles,"alarma",patron)
patron = r'(?i)^portero'
crear_columnas_conpatron(redpiso_detalles,"porteria",patron)
patron = r'(?i)^conserje'
crear_columnas_conpatron(redpiso_detalles,"conserje",patron)
patron = r'(?i)^amueblado'
crear_columnas_conpatron(redpiso_detalles,"amueblado",patron)
patron = r'(?i)^independiente'
crear_columnas_conpatron(redpiso_detalles,"cocina_independiente",patron)
##patron = r'(?i)^cocina equipada'
#crear_columnas_conpatron(redpiso_detalles,"cocina_equipada",patron)
patron = r'(?i)^americana'
crear_columnas_conpatron(redpiso_detalles,"cocina_americana",patron)
patron = r'(?i)^integrada'
crear_columnas_conpatron(redpiso_detalles,"cocina_integrada",patron)
patron = r'(?i)^cocina equipada' 
crear_columnas_conpatron(redpiso_detalles,"cocina",patron)
patron = r'(?i)^plantas:'
crear_columnas_conpatron(redpiso_detalles,"num_plantas",patron)
patron = r'(?i)^pisos:'
crear_columnas_conpatron(redpiso_detalles,"num_pisos",patron)



C:\Users\extas\AppData\Local\Temp\ipykernel_18860\2250767380.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Baños: 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, nueva_col] = row[col]
C:\Users\extas\AppData\Local\Temp\ipykernel_18860\2250767380.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Dormitorios: 2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, nueva_col] = row[col]
C:\Users\extas\AppData\Local\Temp\ipykernel_18860\2250767380.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Armarios: 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, nueva_col] = r

In [747]:
redpiso_detalles.query("referencia=='RP2842024125161'")

,title,precio,descripcion,referencia,cantidad_visitas,fecha_publicacion,m2_constr,campo9,campo10,campo11,campo12,campo13,tiempo2,campo14,campo15,tiempo3,campo16,campo17,campo18,tiempo4,campo19,planta,banos,dormitorios,armarios,consumo_energetico,emisiones_CO2,reformado,obra_nueva,estado,ascensor,sin_ascensor,aire_acondicionado,orientacion,ano_construccion,antiguedad,agua_caliente,calefaccion,tipo_suelo,tipo_fachada,garaje,m2_utiles,exterior,interior,trastero,piscina,zonas,gastos_comunidad,terraza,balcon,jardin,acceso_minusvalido,puerta_blindada,alarma,porteria,conserje,amueblado,cocina_independiente,cocina_americana,cocina_integrada,cocina,num_plantas,num_pisos
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500,"Piso en venta en CALLE BENIMAMET, San Cristóba...",RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años,NaN,Baños: 1,Dormitorios: 2,NaN,NaN,NaN,NaN,NaN,Estado: Perfecto Estado,NaN,NaN,Aire acondicionado,NaN,Año de construcción: 1962,Antigüedad: Más de 50 años,Agua caliente: Individual,NaN,Tipo de suelo: Tarima normal,NaN,NaN,Metros útiles: 60 m²,Exterior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [748]:
redpiso_detalles["calefaccion"].value_counts(dropna=False)

calefaccion
NaN    1468
Name: count, dtype: int64

In [749]:
redpiso_detalles["garaje"].value_counts(dropna=False)

garaje
NaN           1161
Garajes: 1     280
Garajes: 2      22
Garajes: 3       5
Name: count, dtype: int64

In [750]:
redpiso_detalles["campo11"].value_counts(dropna=False)

campo11
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [751]:
def eliminar_no_visitas(texto):
    # Eliminar "No Visitas" y dejar solo el número que le sigue
    return re.sub(r'(?i)No Visitas\s*(\d+)', r'\1', str(texto))

# Aplicar la función a la columna "cantidad_visitas"
redpiso_detalles["cantidad_visitas"] = redpiso_detalles["cantidad_visitas"].str.replace("Nº Visitas ","",case=False,regex=True)
redpiso_detalles["m2_utiles"] = redpiso_detalles["m2_utiles"].str.replace("m2", "", case=False, regex= True)
redpiso_detalles["m2_utiles"] = redpiso_detalles["m2_utiles"].str.replace("Metros útiles:", "", case=False, regex= True)
redpiso_detalles["m2_constr"] = redpiso_detalles["m2_constr"].str.replace("m²", "", case=False, regex= True)
redpiso_detalles["dormitorios"] = redpiso_detalles["dormitorios"].str.replace("Dormitorios:", "", case=False, regex= True)
#redpiso_detalles["banos"] = redpiso_detalles["banos"].str.replace("s", "", case=False, regex= True)
redpiso_detalles["armarios"] = redpiso_detalles["armarios"].str.replace("Armarios: ", "", case=False, regex= True)
redpiso_detalles["banos"] = redpiso_detalles["banos"].str.replace("Baños", "", case=False, regex= True)
redpiso_detalles["antiguedad"] = redpiso_detalles["antiguedad"].str.replace("Antigüedad:", "", case=False, regex= True)
redpiso_detalles["ano_construccion"] = redpiso_detalles["ano_construccion"].str.replace("Año de construcción:", "", case=False, regex= True)
redpiso_detalles["garaje"] = redpiso_detalles["garaje"].str.replace("Garajes:", "", case=False, regex= True)
redpiso_detalles["agua_caliente"] = redpiso_detalles["agua_caliente"].str.replace("Agua caliente:", "", case=False, regex= True)
redpiso_detalles["tipo_suelo"] = redpiso_detalles["tipo_suelo"].str.replace("Tipo de suelo:", "", case=False, regex= True)
redpiso_detalles["tipo_fachada"] = redpiso_detalles["tipo_fachada"].str.replace("Tipo de fachada:", "", case=False, regex= True)
redpiso_detalles["estado"] = redpiso_detalles["estado"].str.replace("Estado:", "", case=False, regex= True)

In [752]:
#renombro campo 10 titular,cammpo 11 detalle, campo 12 son m2_constr y campo 9 dormitorios

redpiso_detalles["detalle"]=redpiso_detalles["campo11"]
redpiso_detalles["titular_anuncio"]=redpiso_detalles["campo10"]

In [753]:
# eliminacion de columnas 
redpiso_detalles= redpiso_detalles.drop(columns=["campo13","tiempo2","campo14","campo15","tiempo3","campo16","campo17","campo18","tiempo4","campo19","campo10","campo11"])

In [754]:
redpiso_detalles["antiguedad"].value_counts(dropna=False)

antiguedad
NaN                     1182
 Más de 50 años          221
 Entre 35 y 50 Años       47
 Mas de 25 Años            8
  Entre 20 y 25 Años       4
 Menos de 5 Años           3
 Entre 10 y  15 Años       1
 Entre 25 y 35 años        1
 Entre 15 y 20 Años        1
Name: count, dtype: int64

In [755]:
redpiso_detalles["ano_construccion"].value_counts(dropna=False)

ano_construccion
NaN      1312
 1965      15
 1960      15
 1930      13
 1940       9
 1972       7
 1964       7
 1970       6
 1962       5
 1925       5
 1966       5
 1959       4
 1950       4
 1900       4
 1955       3
 1920       3
 1968       3
 1961       3
 1969       3
 1979       2
 1994       2
 1923       2
 1910       2
 1957       2
 1967       2
 1929       2
 2000       2
 1986       2
 1973       2
 1914       1
 1977       1
 1976       1
 1990       1
 1945       1
 45         1
 1918       1
 1975       1
 1887       1
 1951       1
 1987       1
 1989       1
 1954       1
 1963       1
 1948       1
 1974       1
 1880       1
 1928       1
 1943       1
 1949       1
 1995       1
 1958       1
Name: count, dtype: int64

In [756]:
 #Usar apply con una función lambda para actualizar antiguedad con ano_construccion cuando sea NaN
redpiso_detalles['antiguedad'] = redpiso_detalles.apply(lambda row: row['ano_construccion'] if pd.isna(row['antiguedad']) else row['antiguedad'], axis=1)

In [757]:
col_tiponuevo= ["precio","m2_constr","dormitorios","armarios"]

redpiso_detalles = redpiso_detalles.replace("nan",0)
redpiso_detalles[col_tiponuevo]=redpiso_detalles[col_tiponuevo].fillna(0)
# Eliminar los separadores de miles solo para los valores mayores a 1.000.000
redpiso_detalles['precio'] = redpiso_detalles['precio'].apply(lambda x: x.replace('.', '') if float(x.replace('.', '')) >= 1000000 else x)
redpiso_detalles[col_tiponuevo] = redpiso_detalles[col_tiponuevo].astype("float")

redpiso_detalles[col_tiponuevo]=redpiso_detalles[col_tiponuevo].replace(0,np.nan)

In [758]:
redpiso_detalles.shape

(1468, 53)

In [759]:
# Reemplazo de valores nulos por ceros y los otros por unos 
#columnas_binario= ["amueblado","alarma","porteria","puerta_seguridad","acceso_minusvalido","aire_acondicionado",
 #                  "ascensor","piscina","trastero","Zonas_verdes"]

#redpiso_detalles[columnas_binario]= redpiso_detalles[columnas_binario].notnull().astype(int)


-----------

In [760]:
redpiso_detalles2= redpiso_detalles
redpiso_normal2= redpiso_normal

In [761]:
redpiso_detalles2.columns

Index(['title', 'precio', 'descripcion', 'referencia', 'cantidad_visitas',
       'fecha_publicacion', 'm2_constr', 'campo9', 'campo12', 'planta',
       'banos', 'dormitorios', 'armarios', 'consumo_energetico',
       'emisiones_CO2', 'reformado', 'obra_nueva', 'estado', 'ascensor',
       'sin_ascensor', 'aire_acondicionado', 'orientacion', 'ano_construccion',
       'antiguedad', 'agua_caliente', 'calefaccion', 'tipo_suelo',
       'tipo_fachada', 'garaje', 'm2_utiles', 'exterior', 'interior',
       'trastero', 'piscina', 'zonas', 'gastos_comunidad', 'terraza', 'balcon',
       'jardin', 'acceso_minusvalido', 'puerta_blindada', 'alarma', 'porteria',
       'conserje', 'amueblado', 'cocina_independiente', 'cocina_americana',
       'cocina_integrada', 'cocina', 'num_plantas', 'num_pisos', 'detalle',
       'titular_anuncio'],
      dtype='object')

In [762]:
redpiso_detalles2["cantidad_visitas"].value_counts(dropna=False)

cantidad_visitas
1002    6
767     5
330     5
373     4
916     4
       ..
128     1
272     1
262     1
441     1
72      1
Name: count, Length: 1087, dtype: int64

In [763]:
redpiso_normal2.columns      #A diferencia del notebook donde se hizo la el trabajo inicialmente aqui tengo la columna officeling_url 

Index(['precio', 'reservado', 'm2_constr', 'dormitorios', 'banos', 'url',
       'title', 'officelink_url', 'nombre_oficina', 'referencia', 'web',
       'fecha_descarga'],
      dtype='object')

In [764]:
redpiso_normal2.isna().sum()

precio             0
reservado          0
m2_constr          3
dormitorios        2
banos             34
url                0
title              0
officelink_url     0
nombre_oficina     0
referencia         0
web                0
fecha_descarga     0
dtype: int64

In [765]:
redpiso_detalles2.shape

(1468, 53)

In [766]:
redpiso_normal2.shape

(1468, 12)

In [767]:
#redpiso_detalles2["dormitorios"] = redpiso_detalles2["dormitorios"].fillna(redpiso_normal2["habitaciones"].reindex(redpiso_normal2.index))

In [768]:
redpiso_detalles2["dormitorios"]= redpiso_detalles2["dormitorios"].astype(str)
redpiso_detalles2["banos"]= redpiso_detalles2["banos"].astype(str)
redpiso_detalles2["m2_constr"]= redpiso_detalles2["m2_constr"].astype(str) 
redpiso_detalles2["armarios"]= redpiso_detalles2["armarios"].astype(str)
redpiso_detalles2["cantidad_visitas"]= redpiso_detalles2["cantidad_visitas"].astype(str)

In [769]:
# redpiso_detalles['fecha_publicacion'] = pd.to_datetime(redpiso_detalles['fecha_publicacion'].astype(str), format='%d%m%Y')

In [770]:
redpiso_detalles2=redpiso_detalles2.dropna(subset=["precio"])
redpiso_normal2= redpiso_normal2.dropna(subset=["precio"])

In [771]:
redpiso_detalles2.shape,redpiso_normal2.shape

((1468, 53), (1468, 12))

In [772]:
#nuevos_nombres={
 #   "descripcion":"titulo",
  #  "cantidad_habitaciones":"cantidad_dormitorios"
#}
#redpiso_normal2= redpiso_normal2.rename(columns=nuevos_nombres)

In [773]:
#redpiso_detalles2["espacio_metros_cuadrados"]=redpiso_detalles2["espacio_metros_cuadrados"].fillna(0) #Tenia dos nulos 
#redpiso_detalles2["dormitorios"]=redpiso_detalles2["cantidad_dormitorios"].fillna(0) #Tenia dos nulos

In [774]:
#redpiso_normal2["precio"]=redpiso_normal2["precio"].astype("float")
#redpiso_detalles2["precio"]=redpiso_detalles2["precio"].astype("float")
#redpiso_detalles2["dormitorios"]=redpiso_detalles2["dormitorios"].astype("int")
#redpiso_detalles2["m2_constr"]=redpiso_detalles2["m2_constr"].astype("float")

In [775]:
redpiso_normal2.head(5)

,precio,reservado,m2_constr,dormitorios,banos,url,title,officelink_url,nombre_oficina,referencia,web,fecha_descarga
0,106.5,1,64.0,2.0,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en CALLE BENIMAMET, San Cristóba...",https://www.redpiso.es/oficina/inmobiliaria-sa...,REDPISO SAN CRISTOBAL,RP2842024125161,redpiso,28/10/2024
1,119.0,0,30.0,1.0,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Usera, Madrid, Madrid",https://www.redpiso.es/oficina/inmobiliaria-us...,REDPISO USERA-ORCASITAS,RP782024130863,redpiso,28/10/2024
2,120.0,1,37.0,1.0,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Buenavista, Carabanchel, Madr...",https://www.redpiso.es/oficina/inmobiliaria-pa...,REDPISO PAU CARABANCHEL,RP442024126007,redpiso,28/10/2024
3,120.0,1,33.0,1.0,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en Buenavista, Carabanchel, Madr...",https://www.redpiso.es/oficina/inmobiliaria-pa...,REDPISO PAU CARABANCHEL,RP442024126427,redpiso,28/10/2024
4,127.5,1,58.0,3.0,1.0,https://www.redpiso.es/inmueble/piso-en-venta-...,"Piso en venta en CALLE BENIFERRI, San Cristóba...",https://www.redpiso.es/oficina/inmobiliaria-sa...,REDPISO SAN CRISTOBAL,RP2842024130186,redpiso,28/10/2024


In [776]:
redpiso_normal2.dtypes

precio            float64
reservado           int64
m2_constr         float64
dormitorios       float64
banos             float64
url                object
title              object
officelink_url     object
nombre_oficina     object
referencia         object
web                object
fecha_descarga     object
dtype: object

In [791]:
url_especifica = 'https://www.redpiso.es/inmueble/piso-en-venta-en-hispanoamerica-chamartin-madrid-madrid-RP902024129170'
redpiso_normal2.loc[redpiso_normal2['url'].str.contains(url_especifica), ['m2_constr', 'planta']] = [155, 1]
redpiso_normal2.loc[redpiso_normal2['url'].str.contains('https://www.redpiso.es/inmueble/en-venta-en-calle-la-alegria-de-la-huerta-los-angeles-villaverde-madrid-madrid-RP332024131222'), ['m2_constr','tiene_piscina','nuda_propiedad']] = [90,0,1]
redpiso_normal2.loc[redpiso_normal2['url'].str.contains('https://www.redpiso.es/inmueble/piso-en-venta-en-madrid-madrid-RP352023120094'), ['m2_constr','tiene_piscina','terraza','trastero']] = [75,0,1,1]
url_especifica = 'https://www.redpiso.es/inmueble/piso-en-venta-en-hispanoamerica-chamartin-madrid-madrid-RP902024129170'
redpiso_normal2.loc[redpiso_normal2['url'].str.contains(url_especifica), ['m2_constr', 'planta']] = [155, 1]


In [792]:
redpiso_normal2.query("m2_constr.isnull()")

,precio,reservado,m2_constr,dormitorios,banos,url,title,officelink_url,nombre_oficina,referencia,web,fecha_descarga,planta,tiene_piscina,nuda_propiedad,terraza,trastero


In [777]:
#redpiso_normal2["espacio_metros_cuadrados"]=redpiso_normal2["espacio_metros_cuadrados"].fillna(0) #Tenia 4 nulos 
#redpiso_normal2["cantidad_dormitorios"]=redpiso_normal2["cantidad_dormitorios"].fillna(0) #Tenia 3 nulos

In [778]:
#redpiso_normal2["precio"]=redpiso_normal2["precio"].astype("float")
#redpiso_normal2["espacio_metros_cuadrados"]=redpiso_normal2["espacio_metros_cuadrados"].astype("float")



In [779]:
redpiso_detalles2.dtypes

title                    object
precio                  float64
descripcion              object
referencia               object
cantidad_visitas         object
fecha_publicacion        object
m2_constr                object
campo9                  float64
campo12                  object
planta                   object
banos                    object
dormitorios              object
armarios                 object
consumo_energetico       object
emisiones_CO2            object
reformado                object
obra_nueva               object
estado                   object
ascensor                float64
sin_ascensor            float64
aire_acondicionado       object
orientacion              object
ano_construccion         object
antiguedad               object
agua_caliente            object
calefaccion             float64
tipo_suelo               object
tipo_fachada             object
garaje                   object
m2_utiles                object
exterior                 object
interior

In [780]:
redpiso_detalles2 = redpiso_detalles2.rename(columns={"título":"titulo"})

In [781]:
redpiso_normal2.columns

Index(['precio', 'reservado', 'm2_constr', 'dormitorios', 'banos', 'url',
       'title', 'officelink_url', 'nombre_oficina', 'referencia', 'web',
       'fecha_descarga'],
      dtype='object')

In [782]:
redpiso_detalles2.shape,redpiso_normal2.shape

((1468, 53), (1468, 12))

In [784]:
redpiso_detalles2['title'].equals(redpiso_detalles2['title']) 

True

In [ ]:
#elimino columnas iguales de los 2 dataframes
#colseliminar=["titulo",'precio']
#redpiso_detalles2.drop(columns=colseliminar,inplace=True)

In [785]:
redpiso_detalles2.columns

Index(['title', 'precio', 'descripcion', 'referencia', 'cantidad_visitas',
       'fecha_publicacion', 'm2_constr', 'campo9', 'campo12', 'planta',
       'banos', 'dormitorios', 'armarios', 'consumo_energetico',
       'emisiones_CO2', 'reformado', 'obra_nueva', 'estado', 'ascensor',
       'sin_ascensor', 'aire_acondicionado', 'orientacion', 'ano_construccion',
       'antiguedad', 'agua_caliente', 'calefaccion', 'tipo_suelo',
       'tipo_fachada', 'garaje', 'm2_utiles', 'exterior', 'interior',
       'trastero', 'piscina', 'zonas', 'gastos_comunidad', 'terraza', 'balcon',
       'jardin', 'acceso_minusvalido', 'puerta_blindada', 'alarma', 'porteria',
       'conserje', 'amueblado', 'cocina_independiente', 'cocina_americana',
       'cocina_integrada', 'cocina', 'num_plantas', 'num_pisos', 'detalle',
       'titular_anuncio'],
      dtype='object')

In [ ]:
redpiso_detalles2["referencia"]

0       REF RP2842024125161
1        REF RP782024130863
2        REF RP442024126007
3        REF RP442024126427
4       REF RP2842024130186
               ...         
2090     REF RP962024124554
2091    REF RP1922024124579
2095     REF RP272023121667
2096    REF RP2532022104355
2097    REF RP2332024131076
Name: referencia, Length: 1468, dtype: object

In [ ]:
# Eliminar "REP" del campo "REF" 
#redpiso_detalles2['referencia'] = redpiso_detalles2['referencia'].str.replace('REF', '').str.strip()

In [ ]:
redpiso_detalles2['referencia']

0       RP2842024125161
1        RP782024130863
2        RP442024126007
3        RP442024126427
4       RP2842024130186
             ...       
2090     RP962024124554
2091    RP1922024124579
2095     RP272023121667
2096    RP2532022104355
2097    RP2332024131076
Name: referencia, Length: 1468, dtype: object

In [ ]:
redpiso_normal2["referencia"]

0       RP2842024125161
1        RP782024130863
2        RP442024126007
3        RP442024126427
4       RP2842024130186
             ...       
2090     RP962024124554
2091    RP1922024124579
2095     RP272023121667
2096    RP2532022104355
2097    RP2332024131076
Name: referencia, Length: 1468, dtype: object

In [ ]:
df_merge= pd.merge(redpiso_detalles2,redpiso_normal2,how="inner",on=["referencia"],indicator=True)

In [ ]:
df_merge.shape

(1470, 48)

In [ ]:
df_merge.duplicated().sum()

0

In [ ]:
df_merge.isna().sum()

referencia                     0
cantidad_visitas               0
fecha_publicacion              0
espacio_metros_cuadrados       2
banos                          7
campo10                      508
campo11                       12
campo12                     1445
dormitorios                   29
cantidad_armarios            884
consumo_energia             1210
emisiones_CO2               1209
cal_energetica               476
estado                       218
antiguedad                  1139
ascensor                       0
aire_acondicionado             0
orientacion                  715
agua_caliente                778
calefaccion                  890
Tipo de suelo_fachada        956
cocina_equipada                0
garaje                      1163
m2_utiles                      2
vista_zona                   342
piscina                        0
trastero                       0
gastos_comunidad            1439
Zonas_verdes                   0
terrazas                    1426
acceso_min

In [ ]:
df_merge.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion',
       'espacio_metros_cuadrados', 'banos', 'campo10', 'campo11', 'campo12',
       'dormitorios', 'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'Tipo de suelo_fachada', 'cocina_equipada', 'garaje', 'm2_utiles',
       'vista_zona', 'piscina', 'trastero', 'gastos_comunidad', 'Zonas_verdes',
       'terrazas', 'acceso_minusvalido', 'puerta_seguridad', 'conserje',
       'num_plantas', 'num_pisos', 'alarma', 'amueblado', 'precio',
       'url_imagen', 'reservado', 'm2_constr', 'cantidad_dormitorios', 'url',
       'titulo', 'contacto', 'officelink_url', 'nombre_oficina', '_merge'],
      dtype='object')

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_merge= df_merge.rename(columns={'campo10': 'titular_anuncio','campo11':'descripcion_extendida','espacio_metros_cuadrados':'m2constr'})

In [ ]:
df_merge["campo12"].value_counts(dropna=False)

campo12
NaN           1445
Adosado         19
Pareado          4
Individual       2
Name: count, dtype: int64

In [ ]:
df_merge['tipologia'] = df_merge['titulo'].str.split().str[0]

In [117]:
df_merge['tipologia'].value_counts(dropna=False)

tipologia
Piso           1335
Ático            38
Apartamento      28
Estudio          22
Casa             19
Dúplex           11
Loft              7
Chalet            7
Venta             3
Name: count, dtype: int64

In [118]:


# Definir una función para concatenar solo si no hay nulos
def concatenar_si_no_nulo(row):
    if pd.notnull(row['tipologia']) and pd.notnull(row['campo12']):
        return row['tipologia'] + ' ' + row['campo12']
    else:
        return row['tipologia']  # O cualquier otro valor que desees en caso de nulos

# Aplicar la función a cada fila del DataFrame
df_merge['tipologia'] = df_merge.apply(concatenar_si_no_nulo, axis=1)




In [119]:
df_merge['tipologia'].value_counts(dropna=False)

tipologia
Piso               1335
Ático                38
Apartamento          28
Estudio              22
Casa Adosado         15
Dúplex               11
Loft                  7
Chalet Adosado        4
Venta                 3
Casa Individual       2
Casa Pareado          2
Chalet Pareado        2
Chalet                1
Name: count, dtype: int64

In [120]:
coleliminar=["imagen4","cantidad_dormitorios","número5",'officelink_url','campo12', 'url_imagen']

In [121]:
redpiso_unido= df_merge

In [122]:
# Creacion de nuevas columnas donde metere los valores de de distrito, barrios, codigos postal y el codigo por distrito y barrio.

nuevas_columnas= ["barrios","cod_barrio","cod_barrio_2","distrito","cod_distrito","codigo_Postal"]

redpiso_unido[nuevas_columnas]=np.nan

In [123]:
redpiso_unido2=redpiso_unido

#### **_Agregacion de barrios y distritos con sus codigos respectivos/ Redpiso._**

In [124]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

In [125]:
# Aqui hacemos la normalizacion de las columnas para despues hacer una mejor filtracion de nombres de distritos y barrios. 



def clasificar_distrito_y_barrio_por_columnas(df_sin_duplicados, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in df_sin_duplicados.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['titulo', 'nombre_oficina']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)
                
                # Verificamos si el valor normalizado contiene el nombre de un distrito o barrio
                for distrito, barrio in zip(distritos, barrios):
                    if distrito in valor_normalizado and pd.isna(fila["distrito"]):
                        df_sin_duplicados.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
                    if barrio in valor_normalizado and pd.isna(fila["barrios"]):
                        df_sin_duplicados.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return df_sin_duplicados

# Llamar a la función pasándole el dataframe y el dataframe de referencia
df_sin_duplicados = clasificar_distrito_y_barrio_por_columnas(redpiso_unido2, dbcp_ordenado)

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1962636392.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'villaverde' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sin_duplicados.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1962636392.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'san cristobal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sin_duplicados.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"


In [ ]:
redpiso_unido2.head(5)

In [127]:
def rellenar_distritos(redpiso_unido2, dbcp_ordenado):
    # Crear un diccionario de barrios a distritos normalizado
    barrio_a_distrito = {normalize_string(barrio): distrito for barrio, distrito in zip(dbcp_ordenado["nombre_barrio"], dbcp_ordenado["nombre_distrito"])}

    for idx, fila in redpiso_unido2.iterrows():
        # Verificamos si el valor de distrito está vacío y si hay un valor en barrios
        if pd.isna(fila["distrito"]) and not pd.isna(fila["barrios"]):
            barrio_normalizado = normalize_string(fila["barrios"])  # Normalizar el valor del barrio
            distrito = barrio_a_distrito.get(barrio_normalizado)  # Buscar el distrito usando el barrio normalizado
            if distrito:
                redpiso_unido2.at[idx, "distrito"] = distrito  # Asignar el distrito correspondiente

# Llamar a la función pasándole el dataframe y el dataframe de referencia
rellenar_distritos(redpiso_unido2=redpiso_unido2, dbcp_ordenado=dbcp_ordenado)


In [128]:

diccionario_ditrito_barrio= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
}


# Función para asignar barrios y distritos recorriendo solo el diccionario
def asignar_barrios_y_distritos(df, diccionario, columnas_busqueda):
    # Normalizar los barrios en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario.items() for barrio in barrios
    }
    
    # Asegurar que las columnas "barrios" y "distritos" existan en el DataFrame
    3#if "barrios" not in df.columns:
     #   df["barrios"] = pd.NA
    #if "distrito" not in df.columns:
     #   df["distrito"] = pd.NA

    # Iterar sobre el diccionario de barrios y distritos
    for barrio_normalizado, distrito in diccionario_normalizado.items():
        # Buscar coincidencias en las columnas especificadas
        for columna in columnas_busqueda:
            # Generar una máscara para las coincidencias
            mask = df[columna].str.contains(barrio_normalizado, na=False, case=False)

            # Asignar valores donde haya coincidencias y columnas vacías
            df.loc[mask & df['barrios'].isna(), 'barrios'] = barrio_normalizado
            df.loc[mask & df['distrito'].isna(), 'distrito'] = distrito

    return df

# Llamar a la función
columnas_busqueda = ["barrios", "distrito"]  # Ajusta estas columnas según tu DataFrame
redpiso_unido2 = asignar_barrios_y_distritos(redpiso_unido2, diccionario_ditrito_barrio, columnas_busqueda)



In [129]:
def asignar_codigos(df_sin_duplicados, dbcp_ordenado):
    """
    Asigna los valores de cod_distrito, cod_barrio, cod_barrio_2 y codigo_postal al DataFrame original
    basándose en la correspondencia con el DataFrame de referencia.
    """
    # Normalizar los nombres en el DataFrame de referencia
    dbcp_ordenado['nombre_distrito_normalizado'] = dbcp_ordenado['nombre_distrito'].apply(normalize_string)
    dbcp_ordenado['nombre_barrio_normalizado'] = dbcp_ordenado['nombre_barrio'].apply(normalize_string)

    # Normalizar las columnas 'distrito' y 'barrios' en el DataFrame original
    df_sin_duplicados['distrito_normalizado'] = df_sin_duplicados['distrito'].apply(normalize_string)
    df_sin_duplicados['barrios_normalizado'] = df_sin_duplicados['barrios'].apply(normalize_string)

    # Realizar asignaciones basadas en coincidencias
    for idx, fila in df_sin_duplicados.iterrows():
        # Verificar coincidencia por distrito
        if pd.notna(fila['distrito_normalizado']):
            coincidencias_distrito = dbcp_ordenado[
                dbcp_ordenado['nombre_distrito_normalizado'] == fila['distrito_normalizado']
            ]
            if not coincidencias_distrito.empty:
                df_sin_duplicados.at[idx, 'cod_distrito'] = coincidencias_distrito['cod_distrito'].iloc[0]

        # Verificar coincidencia por barrio
        if pd.notna(fila['barrios_normalizado']):
            coincidencias_barrio = dbcp_ordenado[
                dbcp_ordenado['nombre_barrio_normalizado'] == fila['barrios_normalizado']
            ]
            if not coincidencias_barrio.empty:
                df_sin_duplicados.at[idx, 'cod_barrio'] = coincidencias_barrio['cod_barrio'].iloc[0]
                df_sin_duplicados.at[idx, 'cod_barrio_2'] = coincidencias_barrio['cod_barrio_2'].iloc[0]
                df_sin_duplicados.at[idx, 'codigo_Postal'] = coincidencias_barrio['codigo_Postal'].iloc[0]

    # Eliminar columnas normalizadas después de asignaciones
    df_sin_duplicados.drop(columns=['distrito_normalizado', 'barrios_normalizado'], inplace=True)
    dbcp_ordenado.drop(columns=['nombre_distrito_normalizado', 'nombre_barrio_normalizado'], inplace=True)

    return df_sin_duplicados

# Llamar a la función con los DataFrames correspondientes
redpiso_unido2 = asignar_codigos(redpiso_unido2, dbcp_ordenado)

In [130]:
redpiso_unido2.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion', 'm2constr',
       'banos', 'titular_anuncio', 'descripcion_extendida', 'campo12',
       'dormitorios', 'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'Tipo de suelo_fachada', 'cocina_equipada', 'garaje', 'm2_utiles',
       'vista_zona', 'piscina', 'trastero', 'gastos_comunidad', 'Zonas_verdes',
       'terrazas', 'acceso_minusvalido', 'puerta_seguridad', 'conserje',
       'num_plantas', 'num_pisos', 'alarma', 'amueblado', 'precio',
       'url_imagen', 'reservado', 'm2_constr', 'cantidad_dormitorios', 'url',
       'titulo', 'contacto', 'officelink_url', 'nombre_oficina', '_merge',
       'tipologia', 'barrios', 'cod_barrio', 'cod_barrio_2', 'distrito',
       'cod_distrito', 'codigo_Postal'],
      dtype='object')

------------------------------------------

In [131]:
# eliminacion de columnas 

redpiso_columnas_eliminar=["url_imagen","nombre_oficina","contacto","officelink_url"]

redpiso_unido2 = redpiso_unido2.drop(columns=redpiso_columnas_eliminar)

In [132]:
redpiso_unido2.shape

(1470, 51)

In [133]:
redpiso_unido2["web"]="redpiso"
redpiso_unido2["fecha_descarga"]="28/10/2024"

In [135]:
redpiso_unido2["Tipo de suelo_fachada"].value_counts()

Tipo de suelo_fachada
Tipo de suelo Gres                  129
Tipo de suelo Parquet               112
Tipo de fachada Ladrillo             70
Tipo de suelo Tarima normal          61
Tipo de suelo Tarima Flotante        53
Tipo de suelo Baldosa                16
Tipo de suelo Sintasol               15
Tipo de suelo Terrazo                14
Tipo de suelo Otros                  12
Tipo de fachada Mono Capa            11
Tipo de fachada Cemento               8
Tipo de suelo Porcelanico             6
Tipo de suelo Mosaico Hidraulico      3
Tipo de fachada Piedra                2
Tipo de fachada Otros materiales      1
Tipo de suelo Marmol                  1
Name: count, dtype: int64

In [136]:
# Aqui vamos a filtrar la informacion de tipo_fachada y la dividire en dos columnas donde se veran los valores de fachada y suelo separados
redpiso_unido2["tipo_fachada"]= np.nan

def asignar_fachada(df):
    # Recorrer cada fila de la columna Tipo de suelo/fachada
    for idx, fila in df.iterrows():
        tipo_suelo_fachada = str(fila['Tipo de suelo_fachada'])  # Asegurarse de que sea un string
        # Convertir el texto a minúsculas para la búsqueda sin importar mayúsculas
        if 'fachada' in tipo_suelo_fachada.lower():
            # Si encuentra la palabra 'fachada', asignar todo el texto a la columna tipo_fachada
            df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada
            # Reemplazar el valor original en Tipo de suelo/fachada por NaN
            df.at[idx, 'Tipo de suelo_fachada'] = np.nan
    return df

# Llamar a la función pasándole tu dataframe
redpiso_unido2 = asignar_fachada(redpiso_unido2)

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\3383542287.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tipo de fachada Ladrillo' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada


In [137]:
redpiso_unido2["Tipo de suelo_fachada"].value_counts()

Tipo de suelo_fachada
Tipo de suelo Gres                  129
Tipo de suelo Parquet               112
Tipo de suelo Tarima normal          61
Tipo de suelo Tarima Flotante        53
Tipo de suelo Baldosa                16
Tipo de suelo Sintasol               15
Tipo de suelo Terrazo                14
Tipo de suelo Otros                  12
Tipo de suelo Porcelanico             6
Tipo de suelo Mosaico Hidraulico      3
Tipo de suelo Marmol                  1
Name: count, dtype: int64

In [138]:
redpiso_unido2["tipo_fachada"].value_counts()

tipo_fachada
Tipo de fachada Ladrillo            70
Tipo de fachada Mono Capa           11
Tipo de fachada Cemento              8
Tipo de fachada Piedra               2
Tipo de fachada Otros materiales     1
Name: count, dtype: int64

In [158]:
# renombramiento de columna 
redpiso_unido2= redpiso_unido2.rename(columns={"Tipo de suelo_fachada":"tipo_suelo"})
redpiso_unido2= redpiso_unido2.rename(columns={"barrios":"barrio"})
redpiso_unido2= redpiso_unido2.rename(columns={"Zonas_verdes":"zonas_verdes"})
redpiso_unido2= redpiso_unido2.rename(columns={"codigo_Postal":"codigo_postal"})

In [159]:
redpiso_unido2.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion', 'banos',
       'titular_anuncio', 'descripcion_extendida', 'campo12', 'dormitorios',
       'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'tipo_suelo', 'cocina_equipada', 'garaje', 'm2_utiles', 'vista_zona',
       'piscina', 'trastero', 'gastos_comunidad', 'zonas_verdes', 'terrazas',
       'acceso_minusvalido', 'puerta_seguridad', 'conserje', 'num_plantas',
       'num_pisos', 'alarma', 'amueblado', 'precio', 'url_imagen', 'reservado',
       'm2_constr', 'cantidad_dormitorios', 'url', 'titulo', 'nombre_oficina',
       'tipologia', 'barrio', 'cod_barrio', 'cod_barrio_2', 'distrito',
       'cod_distrito', 'codigo_postal'],
      dtype='object')

In [160]:
columnas_a_eliminar = ["_merge","officelink_url","contacto","m2constr"]

redpiso_unido2= redpiso_unido2.drop(columns=columnas_a_eliminar)

KeyError: "['_merge', 'officelink_url', 'contacto', 'm2constr'] not found in axis"

In [161]:
redpiso_unido2.to_csv("red_pisoparaunir.csv",index=False)

In [150]:
redpiso_unido2[['url']].value_counts(dropna=False)

url                                                                                                                             
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pinzon-vista-alegre-carabanchel-madrid-madrid-RP302023121517                 4
https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-alberto-aguilera-gaztambide-chamberi-madrid-madrid-RP1782024122328    1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-portalegre-opanel-carabanchel-madrid-madrid-RP2412024124127                  1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-principe-de-vergara-hispanoamerica-chamartin-madrid-madrid-RP902023109627    1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-principe-de-vergara-hispanoamerica-chamartin-madrid-madrid-RP902022102747    1
                                                                                                                                   ..
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-colomer-guin

In [ ]:
dfredpiso['web']='redpiso'
dfredpiso['fecha_descarga']='31/10/2024'
# Llamar a la función pasándole tu dataframe
dfredpiso= asignar_fachada(dfredpiso)
#renombrar titulo a title en redpiso
dfredpiso.rename(columns={'titulo': 'title', 'conserje':'porteria','antiguedad':'ano_construccion','emisiones_CO2':'emisiones_co2'}, inplace=True)
dfredpiso["zona"] = dfredpiso["nombre_oficina"].str.replace("^REDPISO", "", regex=True)
dfunion.rename(columns={'read_all': 'descripcion_extendida'}, inplace=True)
dfi.rename(columns={'read_all': 'descripcion_extendida','acceso_minusv':'acceso_minusvalido'}, inplace=True)
celiminar=['url_imagen','dormitorios']
dfredpiso.drop(columns=celiminar,inplace=True)
dfredpiso.rename(columns={'cantidad_dormitorios': 'dormitorios'}, inplace=True)
celiminar=['title2','ciudad']
dfi.drop(columns=celiminar,inplace=True)

celiminar=['Unnamed: 0']
dfpm.drop(columns=celiminar,inplace=True)


#coleliminar=['salon']
#dfunion.drop(dolumns=celiminar,inplace=True)
dfi['certificado_energetico'] = dfi['letrace'].apply(lambda x: 'Disponible' if pd.notna(x) and x != '' else 'No indicado')
#en la columna antiguedad de redpiso_pisosmadrid2 quitar el texto año Ano de construccion del valor
dfredpiso['ano_construccion']=dfredpiso['ano_construccion'].str.replace('Ano de construccion','')
dfredpiso['ano_construccion']=dfredpiso['ano_construccion'].str.replace('Antiguedad','')
# dfredpiso["plantas"]=dfredpiso['plantas'].str.replace('Plantas','')
# Crear la nueva columna 'exterior' y asignar el valor 1 cuando 'vista zona' sea 'Exterior'
# dfredpiso=dfredpiso.drop("plantas",axis=1)

#dfredpiso['exterior'] = dfredpiso['vista_zona'].apply(lambda x: 1 if x == 'Exterior' else np.nan)
# redpiso_pisosmadrid2=redpiso_pisosmadrid2.drop("vista zona",axis=1)


In [143]:
redpiso_unido2.to_csv("red_pisoparaunir.csv",index=False)